In [14]:
# logging utility

import time
from datetime import datetime

LOG_FILE = "latent_experiment_log_180226.txt"

def log_message(message: str):
    stamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{stamp}] {message}"
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(line + "\n")



In [15]:
# --------------------------------------------------------------
# Latent-Space Face Impersonation (Reduced-Compute Variant)
# MSc Dissertation - Stella Williams
# Inversion-free latent target synthesis experiment
# Multi-run evaluation for repeatability and target specificity
# This run is to get the plot working from file 2_latent_impersonation_stylegan-refactor_NO_INVERSION_Loop2
# --------------------------------------------------------------




In [16]:
import os
import sys
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1


In [17]:
"Cell 2: Device, FaceNet setup"


# --------------------------------------------------------------
# Device configuration
# --------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# --------------------------------------------------------------
# FaceNet embedding model (verification system)
# --------------------------------------------------------------

facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)
# --------------------------------------------------------------
# Preprocessing for FaceNet (160x160 input requirement)
# --------------------------------------------------------------
to_facenet = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

def load_image(path):
    return Image.open(path).convert("RGB")

def img_to_tensor(img):
    return to_facenet(img).unsqueeze(0).to(device)

@torch.no_grad()
def facenet_embed(img_tensor):
    emb = facenet(img_tensor)
    return nn.functional.normalize(emb, p=2, dim=1)

def cosine(a, b):
    return (a * b).sum(dim=1)



Using device: cpu


In [18]:
# --------------------------------------------------------------
# Load pretrained StyleGAN2-ADA generator
# --------------------------------------------------------------
import sys
sys.path.append("/Users/stel/Documents/Dissertation/msc-biometric-security-clean/stylegan2-ada-pytorch")



import pickle

STYLEGAN_REPO = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/stylegan2-ada-pytorch"
STYLEGAN_PKL  = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Models/stylegan2-ffhq.pkl"

if STYLEGAN_REPO not in sys.path:
    sys.path.append(STYLEGAN_REPO)

with open(STYLEGAN_PKL, "rb") as f:
    G = pickle.load(f)["G_ema"].to(device).eval()

print("Loaded StyleGAN2")
print("z_dim:", G.z_dim)





Loaded StyleGAN2
z_dim: 512


In [19]:
# --------------------------------------------------------------
# Generator wrapper
# Converts single w latent into w+ internally
# --------------------------------------------------------------

def synth_from_w_single(w):
    """
    Accepts either:
    - w shape [1, w_dim]
    - w shape [1, num_ws, w_dim] (w+)

    Returns image in [0,1]
    """

    if w.ndim == 2:
        # w is [1, w_dim] → broadcast to w+
        w_plus = w.unsqueeze(1).repeat(1, G.synthesis.num_ws, 1)
    elif w.ndim == 3:
        # Already w+
        w_plus = w
    else:
        raise ValueError(f"Unexpected latent shape: {w.shape}")

    img = G.synthesis(w_plus, noise_mode="const")
    img = (img + 1) / 2
    return img.clamp(0, 1)



In [20]:
# --------------------------------------------------------------
# Load target identity (impersonation objective)
# --------------------------------------------------------------

TARGET_A_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Lenny_Kravitz/Lenny_Kravitz_0001.jpg"
TARGET_B_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Patricia_Clarkson/Patricia_Clarkson_0001.jpg"

targetA_img = load_image(TARGET_A_PATH)
targetB_img = load_image(TARGET_B_PATH)

targetA_t = img_to_tensor(targetA_img)
targetB_t = img_to_tensor(targetB_img)

with torch.no_grad():
    e_target = facenet_embed(targetA_t)
    e_non_target = facenet_embed(targetB_t)

print("Target embedding norm:", e_target.norm().item())

# print("Initial w shape:", w.shape)
#inspecting the shape, it is likely [1, 1, w_dim], or [1, num_ws, w_dim] - with the latter meaning
#we are alreadz in w+ space and the wrapper must not broadcast again. in my previous version, no loop,
#i initialised w differently, mapping for from a random latent or mapping. dimensions changed.
print("Target embedding ready.")

Target embedding norm: 1.0
Target embedding ready.


In [21]:
log_message("--------------------------------------------------")
log_message(f"Experiment started: {datetime.now()}")
log_message("Reduced-compute latent impersonation batch")
log_message("--------------------------------------------------")

In [22]:
MAX_STEPS = 3
LR = 0.03

log_message("--------------------------------------------------")
log_message("Single-run 3-step latent impersonation test")
log_message("--------------------------------------------------")

# Random z initialisation
z = torch.randn(1, G.z_dim, device=device)
with torch.no_grad():
    w = G.mapping(z, None)

w = w.clone().detach().requires_grad_(True)
optimizer = optim.Adam([w], lr=LR)

start_time = time.time()

pbar = tqdm(range(MAX_STEPS), desc="Single Run", leave=True)

for step in pbar:
    img = synth_from_w_single(w)
    img_160 = nn.functional.interpolate(img, (160,160), mode="bilinear", align_corners=False)

    emb = facenet(img_160)
    emb = nn.functional.normalize(emb, dim=1)

    sim_target = cosine(emb, e_target)
    sim_nontarget = cosine(emb, e_nontarget)

    loss = 1 - sim_target.mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    pbar.set_postfix({
        "cos_target": f"{sim_target.item():.4f}",
        "cos_non_target": f"{sim_nontarget.item():.4f}"
    })

    log_message(
        f"Step {step} | "
        f"cos_target={sim_target.item():.4f} | "
        f"cos_non_target={sim_nontarget.item():.4f}"
    )

runtime = time.time() - start_time

final_target = sim_target.item()
final_non_target = sim_nontarget.item()

print("\nFinal Results")
print("Cosine (Target A):", final_target)
print("Cosine (Non-Target B):", final_non_target)
print("Runtime (minutes):", runtime/60)

log_message(
    f"FINAL | cos_target={final_target:.4f} | "
    f"cos_non_target={final_non_target:.4f} | "
    f"runtime_min={runtime/60:.2f}"
)


Single Run:   0%|                                         | 0/3 [00:00<?, ?it/s]


NameError: name 'synth_from_w' is not defined

In [ ]:
with torch.no_grad():
    final_img = synth_from_w(w).detach().cpu()

plt.figure(figsize=(5,5))
plt.imshow(final_img.squeeze().permute(1,2,0))
plt.title(
    f"Single-run latent synthesis\n"
    f"cos_target={final_target:.3f} | cos_non_target={final_non_target:.3f}"
)
plt.axis("off")
plt.show()


In [131]:

log_message("--------------------------------------------------")
log_message(f"Experiment started: {datetime.now()}")
log_message("Reduced-compute latent impersonation batch")
log_message("--------------------------------------------------")


In [132]:
# --------------------------------------------------------------
# Load non-target identity (Target B)
# --------------------------------------------------------------
NON_TARGET_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Patricia_Clarkson/Patricia_Clarkson_0001.jpg"

non_target_img = load_image(NON_TARGET_PATH)
non_target_t = img_to_tensor(non_target_img)

with torch.no_grad():
    e_nontarget = facenet_embed(non_target_t)


In [133]:
# --------------------------------------------------------------
# Multi-Run Latent Impersonation Experiment   
# --------------------------------------------------------------    
NUM_RUNS = 5
MAX_STEPS = 80
TARGET_SIM = 0.85
LR = 0.03

results = []
best_example = None

print("\nRunning latent impersonation batch...\n")
print("NUM_RUNS =", NUM_RUNS)
print("Initial results length =", len(results))


for run in range(NUM_RUNS):
    print("Entered outer loop run", run)


    print(f"\n=== Run {run+1}/{NUM_RUNS} ===")
    log_message(f"\n=== Run {run+1}/{NUM_RUNS} ===")

    torch.manual_seed(run)

    # Initialise latent
    with torch.no_grad():
        w = G.mapping.w_avg.unsqueeze(0).to(device)

    w = w.clone().detach().requires_grad_(True)

    optimizer = optim.Adam([w], lr=0.03)

    start_time = time.time()
    success = False

    for step in tqdm(range(MAX_STEPS), desc=f"Run {run+1}", leave=True):

        img = synth_from_w_single(w)
        img_160 = nn.functional.interpolate(img, (160,160))

        emb = facenet(img_160)
        emb = nn.functional.normalize(emb, dim=1)

        sim_target = cosine(emb, e_target)
        sim_nontarget = cosine(emb, e_nontarget)

        loss = 1 - sim_target.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 10 == 0:
            msg = f"Run {run+1} | Step {step} | cos_target={sim_target.item():.4f} | cos_non_target={sim_nontarget.item():.4f}"
            print(msg)
            log_message(msg)

        if sim_target.item() >= TARGET_SIM:
            success = True
            break

    runtime = time.time() - start_time

    # Final evaluation
    final_target = sim_target.item()
    final_non_target = sim_nontarget.item()

summary = (
    f"Run {run+1} COMPLETE | "
    f"Success={success} | "
    f"Steps={step} | "
    f"Final_target={final_target:.4f} | "
    f"Final_non_target={final_non_target:.4f} | "
    f"Runtime_sec={runtime:.2f}"
)

print(summary)
log_message(summary)

results.append({
    "success": success,
    "steps": step + 1,
    "final_sim_target": final_target,
    "final_sim_non_target": final_non_target,
    "runtime_sec": runtime
})

print("Appended result. Current results length:", len(results))





Running latent impersonation batch...

NUM_RUNS = 5
Initial results length = 0
Entered outer loop run 0

=== Run 1/5 ===


Run 1:   1%|▍                                 | 1/80 [07:05<9:20:04, 425.37s/it]

Run 1 | Step 0 | cos_target=-0.0156 | cos_non_target=0.1060


Run 1:  14%|████▎                          | 11/80 [1:10:28<7:12:37, 376.20s/it]

Run 1 | Step 10 | cos_target=0.7614 | cos_non_target=0.0849


...

Run 5 | Step 0 | cos_target=-0.0156 | cos_non_target=0.1060


Run 5:  14%|████▎                          | 11/80 [1:09:28<7:28:14, 389.77s/it]

Run 5 | Step 10 | cos_target=0.7614 | cos_non_target=0.0849


Run 5:  19%|█████▊                         | 15/80 [1:43:34<7:28:48, 414.29s/it]

Run 5 COMPLETE | Success=True | Steps=15 | Final_target=0.8527 | Final_non_target=0.0180 | Runtime_sec=6214.38
Appended result. Current results length: 1


In [134]:
print("\n==============================")
print("Batch Summary")
print("==============================")

if len(results) == 0:
    print("No runs completed. 'results' list is empty.")
else:
    successes = [r for r in results if r["success"]]
    success_rate = len(successes) / len(results) * 100

    print(f"Total runs completed: {len(results)}")
    print(f"Success rate: {success_rate:.1f}%")

    if len(successes) > 0:
        mean_steps = np.mean([r["steps"] for r in successes])
        mean_target = np.mean([r["final_sim_target"] for r in successes])
    else:
        mean_steps = 0
        mean_target = 0

    mean_non_target = np.mean([r["final_sim_non_target"] for r in results])
    mean_runtime = np.mean([r["runtime_sec"] for r in results]) / 60

    print(f"Mean steps (successful only): {mean_steps:.2f}")
    print(f"Mean final cosine (target): {mean_target:.4f}")
    print(f"Mean cosine (non-target): {mean_non_target:.4f}")
    print(f"Mean runtime: {mean_runtime:.2f} minutes")



Batch Summary
Total runs completed: 1
Success rate: 100.0%
Mean steps (successful only): 16.00
Mean final cosine (target): 0.8527
Mean cosine (non-target): 0.0180
Mean runtime: 103.57 minutes


In [135]:
# --------------------------------------------------------------
# Visualise final generated adversarial face
# --------------------------------------------------------------

if best_example is not None:
    plt.figure(figsize=(5,5))
    plt.imshow(best_example.squeeze().permute(1,2,0))
    plt.title("Example successful latent impersonation")
    plt.axis("off")
    plt.show()

